In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [30]:
print(shakespeare_text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [31]:
vocab = sorted(set(shakespeare_text))
print(vocab)
len(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


65

In [32]:
char_to_ind = {u:i for i, u in enumerate(vocab)}
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [33]:
ind_to_char = np.array(vocab)
ind_to_char


array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [34]:
encoded_text = np.array([char_to_ind[c] for c in shakespeare_text])
encoded_text

array([18, 47, 56, ..., 45,  8,  0])

In [35]:
sample = shakespeare_text[:20]
sample

'First Citizen:\nBefor'

In [36]:
encoded_text[:20]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56])

In [37]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [38]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [39]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [40]:
max_id = len(tokenizer.word_index) # number of distinct character
dataset_size = tokenizer.document_count #total number of characters

In [41]:
print(max_id, dataset_size)

39 1115394


In [42]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [43]:
train_size = dataset_size*90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [44]:
n_steps = 100
window_length = n_steps+1 #target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [45]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [46]:
dataset = dataset.shuffle(10000).batch(32)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

In [47]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [48]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [49]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape = [None, max_id], dropout=0.2), #recurrent_dropout=0.2
    keras.layers.GRU(128, return_sequences=True, dropout=0.2), #recurrent_dropout=0.2
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [50]:
history = model.fit(dataset, epochs=5)

Epoch 1/5
31368/31368 [==============================] - 378s 12ms/step - loss: 1.6262
Epoch 2/5
31368/31368 [==============================] - 379s 12ms/step - loss: 1.5402
Epoch 3/5
31368/31368 [==============================] - 379s 12ms/step - loss: 1.5180
Epoch 4/5
31368/31368 [==============================] - 375s 12ms/step - loss: 1.5071
Epoch 5/5
31368/31368 [==============================] - 374s 12ms/step - loss: 1.5001


In [51]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X, max_id)

In [54]:
X_new = preprocess(['How are yo'])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1] #1st sentence, last char

'u'

In [55]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [56]:
next_char("How are yo", temperature=1)

'u'

In [57]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [63]:
print(complete_text("there", temperature=0.2))

therefore the youngest son where is a scolding of my fa


In [67]:
print(complete_text("long", temperature=0.2))

long the worst.

grumio:
here is a scolding stands to 


In [68]:
print(complete_text("t", temperature=2))

touee,
 do : charguifs; pewrangsicha' hamefay:
quoe


In [69]:
tf.random.set_seed(42)

In [70]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [71]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [72]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [73]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [74]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
      6/Unknown - 2s 54ms/step - loss: 3.6230WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0250s vs `on_train_batch_end` time: 0.0257s). Check your callbacks.
313/313 [==============================] - 8s 21ms/step - loss: 2.6284
Epoch 2/50
313/313 [==============================] - 7s 22ms/step - loss: 2.2383
Epoch 3/50
313/313 [==============================] - 7s 22ms/step - loss: 2.1088
Epoch 4/50
313/313 [==============================] - 7s 21ms/step - loss: 2.0340
Epoch 5/50
313/313 [==============================] - 7s 22ms/step - loss: 1.9836
Epoch 6/50
313/313 [==============================] - 7s 22ms/step - loss: 1.9460
Epoch 7/50
313/313 [==============================] - 7s 22ms/step - loss: 1.9190
Epoch 8/50
313/313 [==============================] - 7s 21ms/step - loss: 1.8980
Epoch 9/50
313/313 [==============================] - 7s 22ms/step - loss: 1.8813
Epoch 10/50
313/313 [======================

For different batch size, we can create stateless copy (remove dropout since it's only used during training)

In [75]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [76]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [77]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [78]:
print(complete_text("t"))

th:
no doing honour this vows want below him not
th


In [81]:
# model.save('shakespeare_model.h5')